In [7]:
import random
import time


class FIFO:
    def __init__(self, capacity):
        self.capacity = capacity
        # Initialize with dummy data to simulate a full FIFO
        self.queue = [1] * capacity

    def is_full(self):
        return len(self.queue) == self.capacity

    def is_empty(self):
        return len(self.queue) == 0

    def enqueue(self, data):
        if not self.is_full():
            self.queue.append(data)

    def dequeue(self):
        if not self.is_empty():
            return self.queue.pop(0)
        return None


class SDRAM:
    def __init__(self, nmin, tsw, trefi, trti):
        self.epsilon = 1e-9  # Tolerance for floating point comparison
        self.nmin = nmin
        self.tsw = tsw
        self.trefi = trefi
        self.trti = trti
        self.data_count = nmin
        self.last_refresh_time = - \
            random.uniform(0, trefi + self.epsilon)  # Random initial state
        self.last_switch_time = 0
        self.is_refreshing = False
        self.is_switching = False

    def operate(self, fifo, current_time):
        # Check if it's time to refresh
        if current_time - self.last_refresh_time > self.trefi \
        or abs(current_time - self.last_refresh_time - self.trefi) < self.epsilon:
            self.is_refreshing = True
            self.last_refresh_time = current_time

        # Refresh in progress
        if self.is_refreshing:
            if current_time - self.last_refresh_time >= self.trti \
                or abs(current_time - self.last_refresh_time - self.trti) < self.epsilon:
                self.is_refreshing = False
            else:
                return  # Stop operation during refresh

        # Check if it's time to switch address
        elif self.data_count == 0 and \
            (current_time - self.last_switch_time >= self.tsw \
            or abs(current_time - self.last_switch_time - self.tsw) < self.epsilon):
            self.is_switching = True

        # Address switch in progress
        if self.is_switching:
            if current_time - self.last_switch_time >= self.tsw - self.epsilon \
                or abs(current_time - self.last_switch_time - self.tsw) < self.epsilon:
                self.is_switching = False
                self.data_count = self.nmin
                self.last_switch_time = current_time
            else:
                return  # Stop operation during address switch

        # Output data to FIFO
        elif self.data_count > 0 and not fifo.is_full():
            fifo.enqueue(1)  # Dummy data
            self.data_count -= 1


class DAC:
    def read_data(self, fifo):
        if fifo.is_empty():
            raise ValueError(
                "DAC tried to read from an empty FIFO. Stopping simulation.")
        return fifo.dequeue()


# Simulation parameters (converted to seconds)
t1 = 6.4e-9  # Time interval for DAC to read from FIFO
t2 = 5e-9    # Time interval for SDRAM to output data to FIFO
Nfifo = 50  # FIFO capacity
Nmin = 85    # SDRAM data count
tSW = 72.5e-9    # SDRAM address switch time
tREFI = 3900e-9  # SDRAM refresh interval
tRTI = 332.5e-9  # SDRAM refresh time

fifo = FIFO(Nfifo)
sdram = SDRAM(Nmin, tSW, tREFI, tRTI)
dac = DAC()

current_time = 0
next_dac_time = t1
next_sdram_time = t2
simulation_duration = 100  # 10 milliseconds

try:
    while current_time < simulation_duration:
        if current_time >= next_dac_time and not fifo.is_empty():
            dac.read_data(fifo)
            next_dac_time += t1

        if current_time >= next_sdram_time:
            sdram.operate(fifo, current_time)
            next_sdram_time += t2

        current_time += 1e-9

except ValueError as e:
    print(e)
    print("Current simulation time:", current_time, "seconds")
else:
    print("FIFO status after 10 milliseconds of simulation:",
          "Empty" if fifo.is_empty() else "Not empty")


KeyboardInterrupt: 